## Setup: Install Dependencies and Import

In [ ]:
# Install required package (run once)
%pip install requests

In [ ]:
# Upload powerbi_admin_api.py to your lakehouse Files folder first!
# Then import it
import sys
sys.path.append('/lakehouse/default/Files')

from powerbi_admin_api import PowerBIAdminAPI
from datetime import datetime, timedelta
import json

## Configuration

In [ ]:
# Configure your Service Principal credentials
TENANT_ID = "your-tenant-id"
CLIENT_ID = "your-app-client-id"
CLIENT_SECRET = "your-app-client-secret"

# Output folder in lakehouse
OUTPUT_FOLDER = "/lakehouse/default/Files/pbi_scans"

# Initialize API client
api = PowerBIAdminAPI(
    tenant_id=TENANT_ID,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET
)

## Helper Function: Save to Lakehouse

In [ ]:
def save_to_lakehouse(data, filename, output_folder=OUTPUT_FOLDER):
    """
    Save data to lakehouse Files folder using mssparkutils.
    Falls back to standard file write if mssparkutils unavailable.
    """
    import json
    
    # Ensure output folder exists
    full_path = f"{output_folder}/{filename}"
    
    try:
        # Try using mssparkutils (Fabric native)
        from notebookutils import mssparkutils
        
        json_content = json.dumps(data, indent=2)
        mssparkutils.fs.put(full_path, json_content, overwrite=True)
        print(f"✅ Saved to lakehouse: {full_path}")
        
    except ImportError:
        # Fallback to standard Python file operations
        # Convert lakehouse path to local path
        local_path = full_path.replace('/lakehouse/default/Files/', '')
        
        with open(local_path, 'w') as f:
            json.dump(data, f, indent=2)
        print(f"✅ Saved to local file: {local_path}")
    
    return full_path

## Example 1: Find Cloud Connections (Single Workspace)

In [ ]:
def find_cloud_connections(workspace_id: str, output_folder=OUTPUT_FOLDER):
    """
    Scan workspace and extract cloud connection information.
    Saves results to lakehouse Files folder.
    """
    print("="*60)
    print(f"SCANNING WORKSPACE FOR CLOUD CONNECTIONS")
    print("="*60)
    
    # Scan workspace with full metadata
    scan_result = api.scan_workspace(
        workspace_ids=workspace_id,
        lineage=True,
        datasource_details=True,
        dataset_schema=True,
        dataset_expressions=True
    )
    
    # Extract datasource instances
    datasources = scan_result.get('datasourceInstances', [])
    print(f"\n✅ Found {len(datasources)} datasource instance(s)\n")
    
    cloud_connections = []
    for ds in datasources:
        ds_type = ds.get('datasourceType', 'Unknown')
        conn_details = ds.get('connectionDetails', {})
        
        print(f"📊 Datasource Type: {ds_type}")
        print(f"   Connection Details: {json.dumps(conn_details, indent=3)}")
        print(f"   Datasource ID: {ds.get('datasourceId')}")
        print(f"   Gateway ID: {ds.get('gatewayId', 'None')}")
        print()
        
        cloud_connections.append({
            'type': ds_type,
            'connectionDetails': conn_details,
            'datasourceId': ds.get('datasourceId'),
            'gatewayId': ds.get('gatewayId')
        })
    
    # Save full scan result to lakehouse
    output_path = save_to_lakehouse(
        scan_result, 
        f"workspace_{workspace_id}_scan.json",
        output_folder
    )
    
    # Save just the cloud connections summary
    save_to_lakehouse(
        cloud_connections,
        f"workspace_{workspace_id}_connections.json",
        output_folder
    )
    
    print("="*60)
    return cloud_connections

# Execute: Replace with your workspace GUID
# workspace_id = "your-workspace-guid-here"
# connections = find_cloud_connections(workspace_id)

## Example 2: Scan All Workspaces for Cloud Connections

In [ ]:
def scan_all_workspaces_for_connections(output_folder=OUTPUT_FOLDER, batch_size=100):
    """
    Scan all workspaces in tenant and extract cloud connections.
    Processes in batches and saves results to lakehouse.
    """
    import time
    
    print("="*60)
    print("SCANNING ALL WORKSPACES FOR CLOUD CONNECTIONS")
    print("="*60)
    
    # Step 1: Get all workspace IDs
    print("\n📋 Retrieving workspace list...")
    workspaces = api.list_workspaces()
    workspace_ids = [ws['id'] for ws in workspaces]
    
    print(f"✅ Found {len(workspace_ids)} workspaces\n")
    
    # Step 2: Scan in batches
    all_connections = []
    batch_results = []
    
    for i in range(0, len(workspace_ids), batch_size):
        batch = workspace_ids[i:i+batch_size]
        batch_num = i//batch_size + 1
        
        print(f"\n🔄 Scanning batch {batch_num}: {len(batch)} workspaces")
        
        try:
            # Scan batch
            scan_result = api.scan_workspace(
                workspace_ids=batch,
                lineage=True,
                datasource_details=True,
                max_wait=900  # 15 minutes for large batches
            )
            
            # Extract datasources from batch
            datasources = scan_result.get('datasourceInstances', [])
            print(f"   ✅ Found {len(datasources)} datasource instances")
            
            # Process each datasource
            for ds in datasources:
                connection_info = {
                    'workspaceId': ds.get('workspaceId'),
                    'datasourceType': ds.get('datasourceType'),
                    'connectionDetails': ds.get('connectionDetails', {}),
                    'datasourceId': ds.get('datasourceId'),
                    'gatewayId': ds.get('gatewayId')
                }
                all_connections.append(connection_info)
            
            # Save batch result
            save_to_lakehouse(
                scan_result,
                f"scan_batch_{batch_num}.json",
                output_folder
            )
            
            batch_results.append({
                'batch': batch_num,
                'workspaces_scanned': len(batch),
                'datasources_found': len(datasources),
                'status': 'success'
            })
            
            # Rate limiting: wait between batches
            if i + batch_size < len(workspace_ids):
                print("   ⏳ Waiting 60 seconds before next batch...")
                time.sleep(60)
                
        except Exception as e:
            print(f"   ❌ Error scanning batch {batch_num}: {e}")
            batch_results.append({
                'batch': batch_num,
                'workspaces_scanned': len(batch),
                'datasources_found': 0,
                'status': 'failed',
                'error': str(e)
            })
            continue
    
    # Save consolidated results
    print("\n📊 Consolidating results...")
    
    save_to_lakehouse(
        all_connections,
        "all_cloud_connections.json",
        output_folder
    )
    
    save_to_lakehouse(
        batch_results,
        "scan_summary.json",
        output_folder
    )
    
    # Summary
    print("\n" + "="*60)
    print("SCAN COMPLETE")
    print("="*60)
    print(f"Total workspaces scanned: {len(workspace_ids)}")
    print(f"Total cloud connections found: {len(all_connections)}")
    print(f"Successful batches: {sum(1 for b in batch_results if b['status'] == 'success')}")
    print(f"Failed batches: {sum(1 for b in batch_results if b['status'] == 'failed')}")
    print("="*60)
    
    return all_connections

# Execute full tenant scan
# all_connections = scan_all_workspaces_for_connections()

## Example 3: Analyze Cloud Connection Types

In [ ]:
def analyze_connection_types(connections_file="all_cloud_connections.json"):
    """
    Analyze cloud connections by type from saved results.
    """
    from notebookutils import mssparkutils
    
    # Load connections from lakehouse
    connections_path = f"{OUTPUT_FOLDER}/{connections_file}"
    content = mssparkutils.fs.head(connections_path, 10000000)  # Read up to 10MB
    connections = json.loads(content)
    
    print("="*60)
    print("CLOUD CONNECTION ANALYSIS")
    print("="*60)
    
    # Count by type
    type_counts = {}
    for conn in connections:
        ds_type = conn.get('datasourceType', 'Unknown')
        type_counts[ds_type] = type_counts.get(ds_type, 0) + 1
    
    # Display sorted by count
    print(f"\nTotal Connections: {len(connections)}\n")
    print("Connection Types:")
    for ds_type, count in sorted(type_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {count:4d} - {ds_type}")
    
    # Save analysis
    analysis = {
        'total_connections': len(connections),
        'types': type_counts,
        'analysis_date': datetime.now().isoformat()
    }
    
    save_to_lakehouse(analysis, "connection_analysis.json", OUTPUT_FOLDER)
    
    print("\n" + "="*60)
    return analysis

# Execute analysis
# analysis = analyze_connection_types()

## Example 4: Quick Test - Single Workspace

In [ ]:
# Quick test with a single workspace
workspace_id = "your-workspace-guid-here"

# Find cloud connections and save to lakehouse
connections = find_cloud_connections(workspace_id)

# Display summary
print(f"\nFound {len(connections)} cloud connections:")
for conn in connections:
    print(f"  - {conn['type']}")